In [ ]:
!conda install --file env.txt --yes

This example requires LAMMPS. You can install it from source or conda via `conda install -c conda-forge lammps`.
You also need to `pip install -e` the topology/gmso package after February 14, 2020, when PR #121 was merged.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import mbuild as mb
import gmso
from gmso.external.convert_mbuild import from_mbuild
from gmso.formats.lammpsdata import write_lammpsdata

In [ ]:
# Generate a small box of Argon atoms using mBuild
ar = mb.Compound(name='Ar')

packed_system = mb.fill_box(
    compound=ar,
    n_compounds=329,
    box=mb.Box([2.5, 2.5, 2.5]),
)

packed_system.visualize()

In [ ]:
# Convert system to a backend object
top = from_mbuild(packed_system)

# Load force field from XML
ff = gmso.ForceField('ar.xml')
ar_type = ff.atom_types['Ar']

# Manually set types, bypassing foyer or any atomtyping engine
for site in top.sites:
    site.atom_type = ar_type

top.update_topology()

# Save files to disk
write_lammpsdata(top, 'data.ar')

Next, we run the simulations (energy minimizaiton and then NVE MD) using LAMMPS

In [ ]:
!lmp_serial < in.argon

Finally we can use use `numpy` and `matplotlib` to do some quick checks of the properties of the system

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data = np.loadtxt('thermo.txt')

In [ ]:
plt.plot(data[:,0], data[:,1])
plt.xlabel('step')
plt.ylabel('Temperature (K)')

In [ ]:
plt.plot(data[:,0], data[:,2])
plt.xlabel('step')
plt.ylabel('Potential Energy (kcal/mol)')